# 4.7 Deriving New Variables

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
path = r'C:\Users\jmowe\OneDrive\5-9-2023 Instacart Basket Analysis'

In [3]:
df_ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [4]:
df_ords_prods_merged.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both


In [5]:
# Create a subset

df = df_ords_prods_merged[:1000000]

In [6]:
#Defining a function

def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'

In [7]:
#Applying function(python suggests using loc function)

df['price_range'] = df.apply(price_label, axis=1)

C:\Users\jmowe\AppData\Local\Temp\ipykernel_20704\2848743372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [10]:
#Checking values

df['price_range'].value_counts(dropna = False)

Mid-range product    756450
Low-range product    243550
Name: price_range, dtype: int64

In [11]:
df['prices'].max()

14.8

In [29]:
#Making a loc function. "If, then" function with the comma being the seperation. Loc function is faster and uses less memory
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [13]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [14]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [15]:
df['price_range_loc'].value_counts(dropna = False)

Mid-range product    756450
Low-range product    243550
Name: price_range_loc, dtype: int64

In [21]:
#Freq of day orders

df_ords_prods_merged['order_day_of_week'].value_counts(dropna = False)

0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: order_day_of_week, dtype: int64

In [25]:
#Creating a for-loop function for days of week

result = []

for value in df_ords_prods_merged["order_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [27]:
df_ords_prods_merged['busiest_day'] = result

In [28]:
df_ords_prods_merged['busiest_day'].value_counts(dropna = False)

Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: busiest_day, dtype: int64

# 4.7 Steps & Answers

### Step 2

In [42]:
# Creating a new shell

result2 = []

In [43]:
# Adding new labels

for value in df_ords_prods_merged['order_day_of_week']:
  if value == 0 or value ==1:
    result2.append('Busiest days')
  elif value == 4 or value == 3:
    result2.append('Slowest days')
  else:
    result2.append('Regular days')

In [44]:
# Defining a new column with results

df_ords_prods_merged['busiest_days'] = result2

In [45]:
df_ords_prods_merged.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,busiest_day,busiest_days
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Regularly busy,Regular days
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Regularly busy,Slowest days
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Regularly busy,Slowest days
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Least busy,Slowest days
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Least busy,Slowest days


### Step 3

In [46]:
df_ords_prods_merged['busiest_days'].value_counts(dropna = False)

Regular days    12916111
Busiest days    11864412
Slowest days     7624336
Name: busiest_days, dtype: int64

In [47]:
#The sum of the 3 regular days in orders is greater than the 2 busiest days, but not by much.

### Step 4

In [49]:
#Figuring out top order hours
hourcounts = df_ords_prods_merged['order_hour_of_day'].value_counts().rename_axis('hour').reset_index(name='counts')

In [50]:
hourcounts

,hour,counts
0,10,2761760
1,11,2736140
2,14,2689136
3,15,2662144
4,13,2660954
5,12,2618532
6,16,2535202
7,9,2454203
8,17,2087654
9,8,1718118


In [52]:
#Creating a for-loop

hours = []

for hour in df_ords_prods_merged['order_hour_of_day']:
  if hour in [10, 11, 14, 15, 13, 12, 16, 9]:
    hours.append('Most orders')
  elif hour in [23, 6, 0, 1, 5, 2, 4, 3]:
    hours.append('Fewest orders')
  else:
    hours.append('Average orders')

In [53]:
df_ords_prods_merged['busiest_period_of_day'] = hours

### Step 5

In [55]:
#Frequency

df_ords_prods_merged['busiest_period_of_day'].value_counts()

Most orders       21118071
Average orders     9997651
Fewest orders      1289137
Name: busiest_period_of_day, dtype: int64

### Step 7

In [56]:
df_ords_prods_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged.pkl'))